# SageMaker Pipeline
--------------

[Amazon SageMaker 모델 구축 파이프라인](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines.html)를 이용하여 ML workflow의 각 단계별 수행을 통해, 모델 학습과 모델 registry에 등록하는 과정에 대한 파이프라인을 만듭니다. 

<p align="center">
<center><img src="./img/mdp_how_it_works.png" height="250" width="850" alt=""><center>
<br><br>
</p>

## 1. import 패키지 설정

In [ ]:
import sagemaker
# import splitfolders

import os
import time
import warnings

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial

import boto3
import numpy as np

# from tqdm import tqdm
from time import strftime

from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

In [ ]:
sess = boto3.Session()
region = sess.region_name
role = get_execution_role()

## 2. 데이터 저장소와 코드 저장소 설정
### 2-1. 데이터 저장소 설정
SageMaker에는 학습에 사용할 데이터 위치와 학습 코드의 위치를 설정합니다. 편의를 위해 default_bucket을 사용했으나, 실제 활용 시에는 이미 생성한 bucket을 활용하는 것도 가능합니다. 

In [ ]:
from pathlib import Path

sess = boto3.Session()
sagemaker_session = sagemaker.Session()
sm = sess.client('sagemaker')
default_bucket = sagemaker_session.default_bucket()

%store default_bucket

### 2-2. 코드 저장소 설정

In [ ]:
iam_client = boto3.client('iam')

role=get_execution_role()
base_role_name=role.split('/')[-1]

iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AWSCodeCommitFullAccess'
)

In [ ]:
codecommit = boto3.client('codecommit')
repository_name = 'informer2020'

try:
    response = codecommit.create_repository(
        repositoryName=repository_name,
        repositoryDescription='Data Scientists share their training code using this Repository'
    )
except:
    
    print("Repository already exists")
    response = codecommit.get_repository(
        repositoryName=repository_name
    )

In [ ]:
code_commit_repo = response['repositoryMetadata']['cloneUrlHttp']
code_commit_repo

In [ ]:
!git init
!git remote add repo_codecommit $code_commit_repo
!git add Informer2020
!git commit -m "Inform2020-update"
!git push repo_codecommit

### 2-3. CodeCommit Credentials 받기

In [ ]:
user_name = 'XXXXX' ## ==> IAM에서 사용자 아이디 확인
codecommit_cred = 'codecommit-cred'

In [ ]:
try:
    response = iam_client.list_service_specific_credentials(
        UserName=user_name,
        ServiceName='codecommit.amazonaws.com'
    )
    if len(response['ServiceSpecificCredentials']) > 0:
        response = iam_client.delete_service_specific_credential(
            UserName=user_name,
            ServiceSpecificCredentialId=response['ServiceSpecificCredentials'][-1]['ServiceSpecificCredentialId']
        )
except:
    print("Create new codecommit crendentials")
    pass
finally:
    response = iam_client.create_service_specific_credential(
        UserName=user_name,
        ServiceName='codecommit.amazonaws.com'
    )
    ServiceUserName = response['ServiceSpecificCredential']['ServiceUserName']
    ServicePassword = response['ServiceSpecificCredential']['ServicePassword']
print(f"ServiceUserName : {ServiceUserName} \nServicePassword : {ServicePassword}")

### 2-4. Secret Manager (Optional)
CodeCommit의 Credentials 정보를 Secret Manager에 Key, Value로 넣어놓고 안전하게 사용합니다.

In [ ]:
iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::aws:policy/SecretsManagerReadWrite'
)

In [ ]:
sec_client = boto3.client('secretsmanager')

In [ ]:
import json
secret_string = json.dumps({
      "username": ServiceUserName,
      "password": ServicePassword
    })

In [ ]:
sec_list = sec_client.list_secrets()['SecretList']

if len(sec_list) == 0:
    sec_response = sec_client.create_secret(
        Name=codecommit_cred,
        Description='This credential uses git_config for SageMaker in Lambda',
        SecretString=secret_string,
        Tags=[
            {
                'Key': 'Name',
                'Value': 'codecommit_credentials'
            },
        ]
    )
else:
    sec_response = sec_client.update_secret(
        SecretId=sec_list[0]['ARN'],
        SecretString=secret_string
    )

In [ ]:
secret_name = sec_response['ARN']
secret_name

## 3. SageMaker Pipelines 라이브러리 import

In [ ]:
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import CreateModelStep

from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.properties import PropertyFile
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.model import Model
from sagemaker.processing import FrameworkProcessor
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.parameters import ParameterInteger, ParameterFloat, ParameterString
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep, JsonGet

## 4. Experiments 관리

Amazon SageMaker에는 실험을 관리할 수 있는 [SageMaker Experiments](https://aws.amazon.com/ko/blogs/aws/amazon-sagemaker-experiments-organize-track-and-compare-your-machine-learning-trainings/) 서비스가 있습니다. 반복적인 실험에 대해 로깅을 남기기 위한 실험 이름 (create_experiment)과 trial (create_trial) 이름을 설정하는 함수입니다. <br> 이러한 메타 정보를 이용하여 향후 ML의 실험 관리가 용이해 질 수 있습니다.

In [ ]:
def create_experiment(experiment_name):
    try:
        sm_experiment = Experiment.load(experiment_name)
    except:
        sm_experiment = Experiment.create(experiment_name=experiment_name,
                                          tags=[{'Key': 'modelname', 'Value': 'informer'}])

In [ ]:
def create_trial(experiment_name, i_type, i_cnt, spot=False):
    create_date = strftime("%m%d-%H%M%s")
    algo = 'informer'
    
    spot = 's' if spot else 'd'
    i_type = i_type[3:9].replace('.','-')
        
    trial = "-".join([i_type,str(i_cnt),algo, spot])
       
    sm_trial = Trial.create(trial_name=f'{experiment_name}-{trial}-{create_date}',
                            experiment_name=experiment_name)

    job_name = f'{sm_trial.trial_name}'
    return job_name

## 5. 실험 설정

학습 시 사용한 소스코드와 output 정보를 저장할 위치를 선정합니다. 이 값은 필수로 설정하지 않아도 되지만, 코드와 결과물을 S3에 저장할 때 체계적으로 정리하는데 활용할 수 있습니다.

In [ ]:
code_location = f's3://{default_bucket}/poc_informer/sm_codes'
output_path = f's3://{default_bucket}/poc_informer/output' 

실험에서 표준 출력으로 보여지는 metrics 값을 정규 표현식을 이용하여 SageMaker에서 값을 capture할 수 있습니다. 이 값은 필수로 설정하지 않아도 되지만, SageMaker Experiments에 Metrics 정보를 남길 수 있어서 실험 관리에 유용합니다.

In [ ]:
metric_definitions = [
    {'Name': 'Epoch', 'Regex': 'Epoch: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?),'},
    {'Name': 'train_loss', 'Regex': 'Train Loss: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?),'},
    {'Name': 'valid_loss', 'Regex': 'Valid Loss: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?),'},
    {'Name': 'test_loss', 'Regex': 'Test Loss: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?),'},
]

다양한 실험 조건을 테스트하기 위해 hyperparameters로 argument 값들을 노트북에서 설정할 수 있으며, 이 값은 학습 스크립트에서 argument인 변수로 받아서 활용이 가능합니다.

In [ ]:
hyperparameters = {
        'model' : 'informer', # model of experiment, options: [informer, informerstack, informerlight(TBD)]
        'data' : 'ETTh1', # data
        'root_path' : 'ETT-small/', # root path of data file
        'data_path' : 'ETTh1.csv', # data file
        'features' : 'M', # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
        'target' : 'OT', # target feature in S or MS task
        'freq' : 'h', # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
        'checkpoints' : 'informer_checkpoints', # location of model checkpoints

        'seq_len' : 96, # input sequence length of Informer encoder
        'label_len' : 48, # start token length of Informer decoder
        'pred_len' : 24, # prediction sequence length
        # Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

        'enc_in' : 7, # encoder input size
        'dec_in' : 7, # decoder input size
        'c_out' : 7, # output size
        'factor' : 5, # probsparse attn factor
        'd_model' : 512, # dimension of model
        'n_heads' : 8, # num of heads
        'e_layers' : 2, # num of encoder layers
        'd_layers' : 1, # num of decoder layers
        'd_ff' : 2048, # dimension of fcn in model
        'dropout' : 0.05, # dropout
        'attn' : 'prob', # attention used in encoder, options:[prob, full]
        'embed' : 'timeF', # time features encoding, options:[timeF, fixed, learned]
        'activation' : 'gelu', # activation
        'distil' : True, # whether to use distilling in encoder
        'output_attention' : False, # whether to output attention in ecoder
        'mix' : True,
        'padding' : 0,
        'freq' : 'h',
        'do_predict' : True,
        'batch_size' : 32,
        'learning_rate' : 0.0001,
        'loss' : 'mse',
        'lradj' : 'type1',
        'use_amp' : False, # whether to use automatic mixed precision training

        'num_workers' : 0,
        'itr' : 1,
        'train_epochs' : 1,  ## Training epochs
        'patience' : 3,
        'des' : 'exp',
        'use_multi_gpu' : True
    }

분산학습과 spot 학습을 사용할지를 선정할 수 있습니다. <br>
분산학습의 경우 [SageMaker data parallel library](https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel.html)를 사용하고자 할 경우 distribution을 아래와 같이 설정한 후 사용할 수 있습니다. 학습 스크립트는 분산 학습 Library로 구현이 필요합니다. 이번 Pipeline에는 분산설정을 추가하지 않았습니다. <br>
[spot 학습](https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html)을 사용하고자 할 경우 학습 파라미터에 spot 파라미터를 True로 변경한 다음, 자원이 없을 때 대기하는 시간인 max_wait (초)를 설정해야 합니다.

In [ ]:
experiment_name = 'informer-poc-exp1'
distribution = None
do_spot_training = True
max_wait = None
max_run = 1*30*60

In [ ]:
instance_type="ml.m5.xlarge"
instance_count=1

## 6. Pipeline parameters, checkpoints와 데이터 위치 설정

[SageMaker Pipeline](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html)의 중요한 기능은 미리 단계를 정의한 다음, 파이프라인의 재정의 없이도 parameters를 실행 중인 단계에서 변경할 수 있다는 것입니다. parameters를 사용하여 이 작업을 수행할 수 있습니다. <br>
ParameterInteger, ParameterFloat, ParameterString를 사용할 수 있으며, 이후 `pipeline.start(parameters=parameters)`를 호출할 때 수정할 수 있는 값을 미리 정의합니다. 특정 parameters만으로 이러한 방식의 정의가 가능합니다.

### 6-1. Pipeline parameters

In [ ]:
train_instance_param = ParameterString(
    name="TrainingInstance",
    default_value="ml.c5.4xlarge",
)

train_count_param = ParameterInteger(
    name="TrainingInstanceCount",
    default_value=1
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)

### 6-2. checkpoints와 데이터 위치 설정

In [ ]:
image_uri = None
train_job_name = 'informer'

if do_spot_training:
    max_wait = max_run

print("train_job_name : {} \ntrain_instance_type : {} \ntrain_instance_count : {} \nimage_uri : {} \ndistribution : {}".format(train_job_name, train_instance_param.default_value, train_count_param.default_value, image_uri, distribution))    



In [ ]:
prefix = 'ETDataset'
inputs = f's3://{default_bucket}/dataset/{prefix}'

source_dir = 'Informer2020'
checkpoint_s3_uri = f's3://{default_bucket}/poc_informer/checkpoints'

### 6-3. Git 설정 (Secret Manager 활용)

In [ ]:
def get_secret(secret_name):
    secret = {}
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager'
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    get_secret_value_response = client.get_secret_value(
        SecretId=secret_name
    )
        
    if 'SecretString' in get_secret_value_response:
        secret = get_secret_value_response['SecretString']
        secret = json.loads(secret)
    else:
        print("secret is not defined. Checking the Secrets Manager")

    return secret

In [ ]:
### CodeCommit의 Credentials이 저장된 secret_name 사용이 필요합니다.
sec_client = boto3.client('secretsmanager')
secret_name = sec_client.list_secrets(SortOrder='desc')['SecretList'][0]['ARN']

In [ ]:
print(f"region : {region}\nsecret_name : {secret_name}\nrole : {role}\ncode_commit_repo : {code_commit_repo}\n")

In [ ]:
secret=get_secret(secret_name)

git_config = {'repo': code_commit_repo,
              'branch': 'main',
              'username': secret['username'],
              'password': secret['password']}


## 7. 학습을 위한 Estimator 선언

AWS 서비스 활용 시 role (역할) 설정은 매우 중요합니다. 이 노트북에서 사용하는 role은 노트북과 training job을 실행할 때 사용하는 role이며, role을 이용하여 다양한 AWS 서비스에 대한 접근 권한을 설정할 수 있습니다.

In [ ]:
create_experiment(experiment_name)
job_name = create_trial(experiment_name, instance_type, instance_count, spot=do_spot_training)

In [ ]:
# all input configurations, parameters, and metrics specified in estimator 
# definition are automatically tracked
estimator = PyTorch(
    entry_point='main_informer.py',
    source_dir=source_dir,
    git_config=git_config,
    role=role,
    sagemaker_session=sagemaker_session,
    framework_version='1.10',
    py_version='py38',
    instance_count=train_count_param,    ## Parameter 값으로 변경
    instance_type=train_instance_param,  ## Parameter 값으로 변경
#     volume_size=256,
    code_location = code_location,
    output_path=output_path,
    hyperparameters=hyperparameters,
    distribution=distribution,
    metric_definitions=metric_definitions,
    max_run=max_run,
    checkpoint_s3_uri=checkpoint_s3_uri,
    use_spot_instances=do_spot_training,  # spot instance 활용
    max_wait=max_wait,
    base_job_name=f"training-{job_name}",
    disable_profiler=True,
    debugger_hook_config=False,
)

## 8. Training 단계 선언

[training 단계](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html)를 사용하여 모델을 학습하는 training job을 생성합니다.<br>
training 단계에는 estimator, training과 validation 데이터 입력 등이 필요합니다. <br>

[caching](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines-caching.html) 를 사용하면 SageMaker 파이프라인이 단계를 실행하기 전에 동일한 인수를 사용하여 호출된 단계의 이전 실행을 찾으려고 시도합니다. 주의할 사항은 파이프라인은 인수가 가리키는 데이터 또는 코드가 변경되었는지 여부를 확인하지 않습니다. 

In [ ]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, 
                           expire_after="7d")

In [ ]:
training_step = TrainingStep(
    name="InformerTrain",
    estimator=estimator,
    inputs={
        "training": sagemaker.inputs.TrainingInput(
            s3_data=inputs
        )
    },
    cache_config=cache_config
)

## 9. Evaluation 단계 - output에서 압축풀어 test_report.json 가져오기

[SageMaker Processing](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep)는 데이터 전처리, 평가 결과의 후처리, 그리고 평가를 수행하는데 활용합니다. <br>
이 단계에서는 Processing job의 예시를 보여주기 위해 압축된 모델 결과에서 압축을 푸는 작업만 수행합니다. processing 단계는 processor, processing 코드를 정의하는 python 스크립트, processing을 위한 output, job 관련 arguments 등으로 구성됩니다.


In [ ]:
framework_processor = FrameworkProcessor(
    PyTorch,
    framework_version="1.10",
    py_version='py38',
    role=role,
    instance_count=1,
    instance_type="ml.c4.xlarge",
    code_location=code_location,
    base_job_name=f"generatingreport-{job_name}",  # choose any name
)

앞선 Train 단계에서의 model 산출물을 postprocessing의 input으로 추가합니다.

In [ ]:
model_input = ProcessingInput(
    source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    destination="/opt/ml/processing/model",
)

In [ ]:
test_report = PropertyFile(
    name="TestReport",
    output_name="result",
    path="test_report.json",
)

In [ ]:
time.sleep(10)
run_args = framework_processor.get_run_args(
    code="postprocess.py",
    source_dir="Informer2020",
    git_config=git_config,
    inputs=[model_input],
    outputs=[
        ProcessingOutput(output_name="result", source="/opt/ml/processing/result")
    ],
    job_name=f"process-step-{job_name}"
)

In [ ]:
postprocessing_step = ProcessingStep(
    name="PostProcessingforInformer",  # choose any name
    processor=framework_processor,
    inputs=run_args.inputs,
    outputs=run_args.outputs,
    code=run_args.code,
    property_files=[test_report],
    cache_config=cache_config
)

## 10. Model 등록 단계

[register model 단계](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.step_collections.RegisterModel)를 사용하여 sagemaker.model.Model 또는 sagemaker.pipeline.PipelineModel을 SageMaker의 model registry에 등록합니다. <br>
PipelineModel은 inference pipeline을 나타내며, inference 요청을 처리하는 container들의 순서를 구성합니다. <br>
register model 단계에서는 등록된 모델의 metrics를 json 구조로 통합하여 등록할 수 있으며, 모델 승인에 대한 방법을 정의할 수 있습니다.

In [ ]:
model_package_group_name = "ts-prediction-informer-test"

In [ ]:
# Register model step that will be conditionally executed
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/test_report.json".format(
            postprocessing_step.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"],
        ),
        content_type="application/json",
    )
)

In [ ]:
register_step = RegisterModel(
    name="InformerRegisterModel",
    estimator=estimator,
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)

## 11. Condition 단계

[condition 단계](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.condition_step.ConditionStep)를 사용하여 단계 properties의 조건을 평가하여 파이프라인에서 다음에 수행할 작업을 진행할지 여부를 판단합니다. <br>

condition 단계에는 condition 목록, condition이 참으로 평가될 경우 실행할 단계 목록, condition이 거짓으로 평가될 경우 실행할 단계 목록 등이 필요합니다. <br>

`[제한사항]`
- SageMaker 파이프라인은 nested condition 단계의 사용을 지원하지 않습니다. 즉, condition 단계를 다른 조건 단계의 입력으로 전달할 수 없습니다.
- condition 단계는 두 개의 분기에서 동일 단계를 사용할 수 없습니다. 즉, 두 개의 분기에서 동일 단계의 기능이 필요한 경우 단계를 복제하고 다른 이름을 지정합니다.

In [ ]:
# Condition step for evaluating model quality and branching execution
cond_lte = ConditionLessThanOrEqualTo(  # You can change the condition here
    left=JsonGet(
        step=postprocessing_step,
        property_file=test_report,
        json_path="regression_metrics.mse.value",  # This should follow the structure of your report_dict defined in the postprocess.py file.
    ),
    right=1.0,  # You can change the threshold here
)
cond_step = ConditionStep(
    name="TestMSECond",
    conditions=[cond_lte],
    if_steps=[register_step],
    else_steps=[],
)

## 12. Pipeline 수행

지금까지 선언한 Step (단계)를 모두 통합합니다. step의 순서는 DAG을 고려하여 자동으로 정의가 됩니다.

In [ ]:
pipeline = Pipeline(
    name="ts-prediction-informer-pipeline",
    parameters=[train_instance_param, train_count_param, model_approval_status],
    steps=[
        training_step,
        postprocessing_step,
        cond_step
    ],
)

SageMaker pipeline 서비스에 정의된 pipeline를 제출하게 됩니다. 기존 정의된 동일한 이름의 pipeline이 있는 경우 덮어쓰기가 됩니다.

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
# Special pipeline parameters can be defined or changed here
parameters = {"TrainingInstance": "ml.c5.4xlarge"}

In [ ]:
start_response = pipeline.start(parameters=parameters)

Pipeline의 진행 사항을 모니터링할 수 있습니다. wait() 함수는 종료가 될 때까지 대기하기 됩니다.

In [ ]:
start_response.wait()
start_response.describe()

<br>
각 step 별로 진행사항을 파악할 수 있습니다.

In [ ]:
start_response.list_steps()

## 13. Model 등록 실행

아래 함수를 이용하여 현재 pending 중인 모델을 확인할 수 있습니다.

In [ ]:
pending_model = sm.list_model_packages(
    ModelPackageGroupName=model_package_group_name,
    ModelApprovalStatus='PendingManualApproval',
    SortBy='Name',
    SortOrder='Descending'
)
pending_model

<br>
아래 명령어를 이용하여 실제 승인을 수행하게 됩니다. 아래 예시는 가장 최근 등록된 버전의 모델을 가져와서 승인하도록 만들었습니다.

In [ ]:
try:
    model_package_update_input_dict = {
    "ModelPackageArn" : pending_model['ModelPackageSummaryList'][0]['ModelPackageArn'],
    "ModelApprovalStatus" : "Approved"
    }
    model_package_update_response = sm.update_model_package(**model_package_update_input_dict)
except:
    print("승인된 모델이 없습니다.")

In [ ]:
model_package_update_response

승인된 모델 중에서 가장 최신 버전의 모델을 검색합니다.

In [ ]:
approved_model = sm.list_model_packages(
    ModelPackageGroupName=model_package_group_name,
    ModelApprovalStatus='Approved',
    SortBy='Name',
    SortOrder='Descending'
)

In [ ]:
model_pkg = sm.describe_model_package(ModelPackageName=approved_model['ModelPackageSummaryList'][0]['ModelPackageArn'])

## 14. SageMaker Model 생성
승인된 모델 정보를 이용하여 들어오는 데이터에 대한 전/후처리가 포함된 predictor.py 파일을 포함하여 SageMaker 모델을 생성합니다.
predictor.py의 전처리는 전체 데이터셋 (CSV)에서 학습에 활용하지 않은 테스트용 데이터셋을 짤라 Ground Truth 값을 제거하고 모델 입력 형태로 변환하는 작업을 합니다. 후처리는 예측된 결과와 Ground Truth 값을 다시 붙여서 결과 CSV로 S3에 저장하도록 구현하였습니다. 


향후 활용시에는 각각의 데이터셋과 모델 환경에 맞게 predictor.py를 구현하여 활용하시면 됩니다.

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

In [ ]:
model = PyTorchModel(
    entry_point='predictor.py',
    source_dir='Informer2020',
    git_config=git_config,
    code_location=code_location,
    model_data=model_pkg['InferenceSpecification']['Containers'][0]['ModelDataUrl'],
    role=role,
    framework_version="1.10",
    py_version="py38"
)

## 14. SageMaker batch transform job 수행
앞에서 생성한 model을 이용하여 데이터셋 (CSV)의 위치가 있는 S3 URI와 입력되는 데이터의 형태, 결과 파일이 저장되는 S3 URI, 이 작업을 수행하는 인스턴스 타입과 개수를 지정하여 예측을 수행합니다.

In [ ]:
transformer= model.transformer(
    instance_count=1,
    instance_type='ml.m5.xlarge',
    assemble_with="Line",
    output_path=f"s3://{default_bucket}/poc_informer/batch_result",
    env={'default_bucket': default_bucket}
)

In [ ]:
from time import strftime

job_name=model_package_group_name+"-"+strftime("%m%d-%H%M%s")

batch_transform = transformer.transform(
    data=f's3://{default_bucket}/dataset/ETDataset/ETT-small/ETTh1.csv',
    data_type='S3Prefix',
    content_type='text/csv',
    split_type='Line',
    job_name=f"tranform-{job_name}",
    wait=False
)

In [ ]:
transformer.wait()